In [6]:
import lxml.etree as etree
import re
from datetime import datetime

In [7]:
tree = etree.parse('/home/leey/Documents/Data/19wahlperiode/19001-data.xml')
#etree.strip_tags(tree, 'inline')
root = tree.getroot()

In [33]:
for child in tree.xpath('sitzungsverlauf'):
    for grandchild in child:
        print(grandchild)

<Element sitzungsbeginn at 0x7f25080a58c8>
<Element tagesordnungspunkt at 0x7f25080a5908>
<Element tagesordnungspunkt at 0x7f25083d1cc8>
<Element tagesordnungspunkt at 0x7f25083d1d48>
<Element tagesordnungspunkt at 0x7f25080a58c8>
<Element tagesordnungspunkt at 0x7f25083d1cc8>
<Element tagesordnungspunkt at 0x7f25083d1d48>
<Element sitzungsende at 0x7f25080a58c8>


### Metadata

In [3]:
# wp under wahlperiode, session under sitzung-nr, date in sitzung-datum, all in dbtplenarprotokoll
for textinfo in tree.xpath("//dbtplenarprotokoll"):
    wp = textinfo.get("wahlperiode")
    session = textinfo.get("sitzung-nr")
    date = textinfo.get("sitzung-datum")

print("Parliament {}/{}".format(wp, session))
print(date) # date as text or datetime object?, currently d-m-Y

Parliament 19/1
24.10.2017


In [39]:
tree.xpath("//dbtplenarprotokoll/@sitzung-datum")

['24.10.2017']

In [45]:
wp = tree.xpath('vorspann/kopfdaten/plenarprotokoll-nummer/wahlperiode/text()')
session = tree.xpath('vorspann/kopfdaten/plenarprotokoll-nummer/sitzungsnr/text()')
for datum in tree.xpath('vorspann/kopfdaten/datum'):
    date = datum.get("date")
print(date)

datetime.strptime(date, '%d.%m.%Y')

24.10.2017


datetime.datetime(2017, 10, 24, 0, 0)

In [38]:
date = tree.xpath('vorspann/kopfdaten/datum/')

XPathEvalError: Invalid expression

### Texts

In [8]:
divs = tree.findall("//rede")

In [22]:
divs_to = tree.findall("//tagesordnungspunkt")

In [5]:
for div in divs:
    for uts in div:
        print("{}: {}".format(uts.tag, uts.text))
        print("######")

p: 

        
######
p: Guten Morgen, liebe Kolleginnen und Kollegen! Nehmen Sie bitte Platz.
######
p: Meine sehr verehrten Damen und Herren! Liebe Kolleginnen und Kollegen! Ich begrüße Sie zur konstituierenden Sitzung des 19. Deutschen Bundestages. Es entspricht der ständigen Übung, zu Beginn der konstituierenden Sitzung nach den Regelungen der bisherigen Geschäftsordnung des Deutschen Bundestages zu verfahren.
######
p: § 1 Absatz 2 der Geschäftsordnung des Deutschen Bundestages sieht vor, dass das am längsten dem Bundestag angehörende Mitglied, das hierzu bereit ist, den Vorsitz übernimmt, bis der Deutsche Bundestag einen Präsidenten gewählt hat.
######
p: Die Fraktion der AfD widerspricht diesem Verfahren und hat auf Drucksache 19/2 beantragt, einen Versammlungsleiter zu wählen, der die konstituierende Sitzung eröffnen soll. Über diesen Antrag lasse ich sofort abstimmen. Wer dem Antrag der Fraktion der AfD zustimmt, den bitte ich um sein Handzeichen. – Gegenstimmen? –
######
komme

In [25]:
# divs_to as tagesordnungspunkt 
i = 0
for div in divs_to:
    for uts in div.iter('rede'):
        i += 1
        for c in uts.iter():
            if c.tag == "p" and c.get("klasse") != "redner":
                print("{}: {}".format(c.tag, c.text))
            elif c.tag == "p" and c.get("klasse") == "redner":
                print("SPEAKER")
            elif c.tag == "kommentar":
                print("{}: {}".format(c.tag, c.text))
            print(i)

1
SPEAKER
1
1
1
1
1
1
1
1
p: Guten Morgen, liebe Kolleginnen und Kollegen! Nehmen Sie bitte Platz.
1
p: Meine sehr verehrten Damen und Herren! Liebe Kolleginnen und Kollegen! Ich begrüße Sie zur konstituierenden Sitzung des 19. Deutschen Bundestages. Es entspricht der ständigen Übung, zu Beginn der konstituierenden Sitzung nach den Regelungen der bisherigen Geschäftsordnung des Deutschen Bundestages zu verfahren.
1
p: § 1 Absatz 2 der Geschäftsordnung des Deutschen Bundestages sieht vor, dass das am längsten dem Bundestag angehörende Mitglied, das hierzu bereit ist, den Vorsitz übernimmt, bis der Deutsche Bundestag einen Präsidenten gewählt hat.
1
p: Die Fraktion der AfD widerspricht diesem Verfahren und hat auf Drucksache 19/2 beantragt, einen Versammlungsleiter zu wählen, der die konstituierende Sitzung eröffnen soll. Über diesen Antrag lasse ich sofort abstimmen. Wer dem Antrag der Fraktion der AfD zustimmt, den bitte ich um sein Handzeichen. – Gegenstimmen? –
1
kommentar: (Lachen b

In [15]:
# divs as rede 
i = 0
for uts in divs: # p in rede
    i += 1
    for c in uts.iter(): # iterate over p 
        if c.tag == "p" and c.get("klasse") != "redner":
            print("{}: {}".format(c.tag, c.text))
        elif c.tag == "p" and c.get("klasse") == "redner":
            print("{}: REDNER".format(c.tag, c.text))
        elif c.tag == "kommentar":
            print("{}: {}".format(c.tag, c.text))
        print(i)

1
p: REDNER
1
1
1
1
1
1
1
1
p: Guten Morgen, liebe Kolleginnen und Kollegen! Nehmen Sie bitte Platz.
1
p: Meine sehr verehrten Damen und Herren! Liebe Kolleginnen und Kollegen! Ich begrüße Sie zur konstituierenden Sitzung des 19. Deutschen Bundestages. Es entspricht der ständigen Übung, zu Beginn der konstituierenden Sitzung nach den Regelungen der bisherigen Geschäftsordnung des Deutschen Bundestages zu verfahren.
1
p: § 1 Absatz 2 der Geschäftsordnung des Deutschen Bundestages sieht vor, dass das am längsten dem Bundestag angehörende Mitglied, das hierzu bereit ist, den Vorsitz übernimmt, bis der Deutsche Bundestag einen Präsidenten gewählt hat.
1
p: Die Fraktion der AfD widerspricht diesem Verfahren und hat auf Drucksache 19/2 beantragt, einen Versammlungsleiter zu wählen, der die konstituierende Sitzung eröffnen soll. Über diesen Antrag lasse ich sofort abstimmen. Wer dem Antrag der Fraktion der AfD zustimmt, den bitte ich um sein Handzeichen. – Gegenstimmen? –
1
kommentar: (Lachen

In [10]:
for div in divs: # rede in all rede
    for uts in div.getchildren(): # p in rede
        if c.tag == "p" and c.get("klasse") != "redner":
            print("{}: {}".format(c.tag, c.text))
        elif c.tag == "p" and c.get("klasse") == "redner":
            print("{}: REDNER".format(c.tag, c.text))
        elif c.tag == "kommentar":
            print("{}: {}".format(c.tag, c.text))

In [ ]:
for c in uts.iter():
                    if self.v > 1:
                        print("{}: {}".format(c.tag, c.text))
                    
                    if c.tag == "p" and c.get("klasse") != "redner":
                        if c.text:
                            text.append(c.text.strip())

                    elif c.tag == "stage":
                        if text:
                            para = self.create_paragraph(text, ut)
                            text = []
                        self.add_interjections(c.text, para)
                    else:
                        print("unknown tag")
                if text:
                    para = self.create_paragraph(text, ut)

### Speaker

In [53]:
for div in divs: # <rede> in [all <rede>]
    for uts in div: # <p> in <rede>
        if uts.get("klasse") == "redner": # <p>
            for redner in uts: # <redner>
                vorname = redner.xpath('name/vorname/text()')
                nachname = redner.xpath('name/nachname/text()')
                fullname = vorname + nachname
                if len(fullname) > 1:
                    fullname = fullname[0] + ' ' + fullname[1]
                if fullname != []:
                    print(fullname)

Alterspräsident Dr. Hermann Otto Solms
Alterspräsident Dr. Hermann Otto Solms
Carsten Schneider
Bernd Baumann
Jan Korte
Michael Grosse-Brömer
Marco Buschmann
Britta Haßelmann
Volker Kauder
Alterspräsident Dr. Hermann Otto Solms
Wolfgang Schäuble
Alterspräsident Dr. Hermann Otto Solms
Wolfgang Schäuble
Dr. Wolfgang Schäuble
Hans-Peter Friedrich
Thomas Oppermann
Wolfgang Kubicki
Petra Pau
Claudia Roth
Dr. Wolfgang Schäuble


In [79]:
for div in divs_to: 
    for uts in div.iter('rede'): # <p> in <rede>
        for redner in uts.xpath("p[@klasse='redner']/redner"):
            vorname = redner.xpath('name/vorname/text()')
            nachname = redner.xpath('name/nachname/text()')
            fullname = vorname + nachname
            if len(fullname) > 1:
                fullname = fullname[0] + ' ' + fullname[1]
            if fullname != []:
                print(fullname)

Alterspräsident Dr. Hermann Otto Solms
Alterspräsident Dr. Hermann Otto Solms
Carsten Schneider
Bernd Baumann
Jan Korte
Michael Grosse-Brömer
Marco Buschmann
Britta Haßelmann
Volker Kauder
Alterspräsident Dr. Hermann Otto Solms
Wolfgang Schäuble
Alterspräsident Dr. Hermann Otto Solms
Wolfgang Schäuble
Dr. Wolfgang Schäuble
Hans-Peter Friedrich
Thomas Oppermann
Wolfgang Kubicki
Petra Pau
Claudia Roth
Dr. Wolfgang Schäuble


### Speaker Metadata

info needed for find_person_in_db.py: <br>
name, position (rolle_lang), party (fraktion), wp (wahlperiode), session (sitzungnr) <br>
name_id with redner_id ? [might be difficult if speaker has already been added before]

## Building a Scraper

In [1]:
import sys
import os

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V
from django.core import serializers

### import serialized data for parliamentarians 

In [18]:
with open("german_parties.xml", "r") as data:
    for obj in serializers.deserialize("xml", data):
        obj.save()

In [19]:
with open("german_parliamentarians.xml", "r") as data:
    for obj in serializers.deserialize("xml", data):
        obj.save()

In [26]:
cmodels.Country.objects.get(name="Germany").id

2921044

In [27]:
# check for parliamentarians
pm.Person.objects.filter(active_country=cmodels.Country.objects.get(name="Germany").id)

<QuerySet [<Person: Dr. Manfred Abelein>, <Person: Dr. Ernst Achenbach>, <Person: Annemarie Ackermann>, <Person: Dr. Else Ackermann>, <Person: Ulrich Adam>, <Person: Rudolf Adams>, <Person: Raban Adelmann>, <Person: Dr. Konrad Adenauer>, <Person: Brigitte Adler>, <Person: Eduard Adorno>, <Person: Dr. Jochen Aerssen>, <Person: Christa Nikkels>, <Person: Willi Agatz>, <Person: Conrad Ahlers>, <Person: Adolf Ahrens>, <Person: Hermann Ahrens (Salzgitter)>, <Person: Dr. Karl Ahrens>, <Person: Dr. Heinrich Aigner>, <Person: Siegbert Alber>, <Person: Johannes Albers>, '...(remaining elements truncated)...']>

In [19]:
pm.Person.objects.filter(surname="Gauland")[0].id

68692

In [3]:
pm.Person.objects.filter(surname="Schulze")

<QuerySet [<Person: Svenja Schulze>, <Person: Gerhard Schulze (Berlin)>, <Person: Waldemar Schulze (Berlin)>, <Person: Frederick Schulze (Sangerhausen)>, <Person: Dr. Klaus-Peter Schulze>, <Person: Frederik Schulze>, <Person: Schulze>]>

In [21]:
pm.Person.objects.get(id=68692).party

In [33]:
pm.Person.objects.filter(positions="{Alterspräsident}")

<QuerySet []>

In [39]:
pm.Person.objects.filter(surname="Merkel")[0].positions

['parliamentarian']

## Inspecting scraper results

In [56]:
import numpy as np

In [89]:
# Deleting bad scraper runs
pm.Document.objects.filter(id=164).delete()

(1, {'parliament.Document_search_matches': 0, 'parliament.Document': 1})

In [5]:
pm.Utterance.objects.filter(document__id=6423)

<QuerySet [<Utterance: Utterance object (1807547)>, <Utterance: Utterance object (1807546)>, <Utterance: Utterance object (1807545)>, <Utterance: Utterance object (1807544)>, <Utterance: Utterance object (1807543)>, <Utterance: Utterance object (1807542)>, <Utterance: Utterance object (1807541)>, <Utterance: Utterance object (1807540)>, <Utterance: Utterance object (1807539)>, <Utterance: Utterance object (1807538)>, <Utterance: Utterance object (1807537)>, <Utterance: Utterance object (1807536)>, <Utterance: Utterance object (1807535)>, <Utterance: Utterance object (1807534)>, <Utterance: Utterance object (1807533)>, <Utterance: Utterance object (1807532)>, <Utterance: Utterance object (1807531)>, <Utterance: Utterance object (1807530)>, <Utterance: Utterance object (1807529)>]>

In [11]:
pm.Paragraph.objects.filter(utterance__id=1807535)[3].text

'Auch Die Linke hat ihre bewährte Rolle. Sie gibt hier die Scheinheilige. Herr Korte trägt vor, es ginge darum, das Parlament schnell arbeitsfähig zu machen. Sie kennen die Praxis des Parlamentarismus in Deutschland ganz genau. Wir wissen heute nicht, welche Ausschüsse es insgesamt geben wird, wie groß sie sein werden, welche Kollegen wohin gehen müssen. Dass das in den Fraktionen natürlich besprochen werden muss, ist Ihnen bekannt.'

In [3]:
pm.Document.objects.last().id

6423

In [14]:
pm.Document.objects.get(id=6423).parlperiod

<ParlPeriod: Germany - National - 19>

In [17]:
pm.Document.objects.filter(parlperiod__n=19)

<QuerySet [<Document: Plenarprotokoll, 19/1, 2017-10-24>]>